In [2]:
from openai import OpenAI
import pandas as pd
#from transformers import pipeline
from dotenv import load_dotenv
import os
from random import randint
#from transformers import AutoModel,AutoTokenizer


In [ ]:
# 

def clean_sentence(sentence):
    # Remove quotation marks and similar characters
    cleaned = sentence.replace('"', '').replace("'", "")
    return cleaned.strip()

# Load API key from .env file
def get_credentials():
    load_dotenv()
    api_key = os.getenv('OPENAI_API_KEY')
    return api_key   

    
    
def generate_swedish_sentences(entity_list, entities_metadata, num_sentences, sentence_start):
    
    completion = client.chat.completions.create(
        temperature=1.0,
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Du är en hjälpsam assistent som genererar spontana, varierade, och autentiska svenska meningar."},

            {
                "role": "user",
                "content": (
                    f"Generera en kort, spontan och vardaglig text på svenska ( max {num_sentences} mening) "
                    f"Meningarna skall innehåll följande entititer {entity_list}, vidare finns en beskrivning av varje entitet: {entities_metadata}."
                    "Meningarna skall vara likt meningar som kan påträffas i en nyhetsartikel."
                    "Antingen tar du rollen som en journalist som läser upp fakta, eller som en person som blir intervjuad"
                    "För kontext finns följande exempelmeningar, med entiteter:"
                    "MENING: Peter Bowin i Ronneby har blivit stalkad i nio år, ENTITETER: Bowin, Peter, Ronneby"
                    "MENING: Islamiska Shiasamfunden har förutom Al Rassol även två andra Blekingeföreningar som medlemmar ., ENTITETER: Shiasamfunden, Islamiska"
                    "MENING: Det handlar om ytterligare en i Karlskrona och en i Karlshamn ., ENTITETER: Karlshamn, Karlskrona"    
                    "MENING: Det ska inte ske att det lämnas ut , säger Anders Johansson , handläggare på polisen. ENTITETER: Johansson, Anders"
                    "Du uppmuntras även att använda dig av entiteterna som en professionell journalist skulle göra."
                    "Tonaliteten skall vara neutral och saklig. Texterna skall ha ett nyhetsvärde, alltså inte vara reklam"
                    "Exempel: ENTITET: Ideella föreningen Cortina Line med firma Cortina Line bör bara benämnas som Cortina Line"   
                    
                )
            }
        ]
    )

    raw_sentence = completion.choices[0].message.content
    cleaned_sentence = clean_sentence(raw_sentence)
    print(cleaned_sentence)
    return cleaned_sentence


client = OpenAI(api_key=get_credentials())

In [15]:
entity_df = pd.read_csv('./entities/manual/entity_df.csv')
company_names_to_ignore = ["Aktiebolaget", "Handelsbolaget", "Aktiebolag", "Handelsbolag", "Filial", "filial", "Firma", "AB", "HB", "AB", "KB", "HB", "AB", "KB", "AB", "HB", 'Handelbola get', 'handelsbolag', 'aktiebolag', 'handelsbolaget', 'aktiebolaget']

sentence_starters = [
    "Jag har en fråga om",
    "Jag undrar om",
    "Kan du berätta om",
    "Vet du något om",
    "Har du koll på",
    "Jag såg nyligen att",
    "Har du hört något om",
    "Vad tycker du om",
    "Fan vad trött man är på",
    "Det är riktigt nice med",
    "Problemed med det är att",
    "Jag läste att",
    "Vet du vart man kan hitta",
    "Jag funderar på",
    "Det vore kul att testa",
    "Ja, jag ska kolla upp",
    "Har du hört något nytt om",
    "Kan du rekommendera",
    "Jag skulle behöva tips på",
    "Tror du det går att fixa",
    "Vet du var man hittar",
    "Det verkar spännande med",
    "Jag är nyfiken på",
    "Är du insatt i",
    "Hur är det med",
    "Har du erfarenhet av",
    "Jag läste precis om",
    "Har du varit på",
    "Jag funderar på att prova",
    "Man borde egentligen testa",
    "Känner du någon som",
    "En polare till mig var",
    "En kille jag känner sa att",
    "En tjejkompis till mig var så less på",
    "Jag fattar inte hur man använder",
    "Vad är grejen med",
    "Vad har vi på",
    "Jag är sjukt taggad på",
    "Kan du berätta lite om",
    "Det vore intressant att höra",
    "Jag behöver veta mer om",
    "Jag fattar inte hypen med",
    "Snälla hjälp mig förstå",
    "Finns det någon särskild anledning att",
    "Vad hände ens med",
    "Kan man använda",
    "Jag är lite osäker på",
    "Jag skulle vilja veta mer om",
    "Har du någonsin använt",
    "Vad är din åsikt om",
    "Vad tycker du om",
    "Tror du att man kan lita på",
    "Kan man lita på",
    "Han tycker att",
    "Hon tycker att",
    "Det är helt galet att",
    "Det är rätt sjukt att",
    "Hon tyckte inte",
    "Jonas tyckte",
    "Problemet är att",
    "Det är svårt att",
    "Det är enkelt att",
    "Det är inte så svårt att",
    "Det är inte så enkelt att",
    "Det är inte så kul att",
    "Det är inte så roligt att",
    "Det är inte så jobbigt att",
    "Det är inte så farligt att",
    "Det är inte så illa att",
    "När man tänker efter",
    "En annan grej är att",
    "Som grädden på moset",
    "Tror du inte att",
    "Det är ju så att",
    "Det är så att",
    "Det är ju verkligen så att",
    "Folk är ganska trötta på",
    "En man i 20-års åldern",
    "En kvinna i 30-års åldern",
    "En man i 40-års åldern",
    "En kvinna i 50-års åldern",
]

In [25]:
num_samples = 10

sentences = []
for i in range(num_samples):
    entity_list = []
    entities_metadata = []
    for j in range(1):
        entity_picker = randint(0, len(entity_df) - 1)
        entity = entity_df.iloc[entity_picker]
        entity_name = entity["entitet"]

        # Remove company prefixes
        for ignore_name in company_names_to_ignore:
            entity_name = entity_name.replace(ignore_name, "")
        
        entity_list.append(entity_name)
        entities_metadata.append({"entity": {entity_name}, "description": entity["beskrivning"], "source": "custom"}) 
        
    print(entity_list)
    print(entities_metadata)    
    
    sentence_start = sentence_starters[randint(0, len(sentence_starters) - 1)]
    num_sentences = 1
    sentence = generate_swedish_sentences(entity_list, entities_metadata, num_sentences, sentence_start)
    sentences.append({"sentence": sentence, "entities": entity_list, "metedata": entities_metadata})
    

    

    

['"CCB Mode eftr. ']
[{'entity': {'"CCB Mode eftr. '}, 'description': '"Bolaget skall idka försäljning av damkläder samt därmed förenlig verksamhet."', 'source': 'custom'}]
CCB Mode eftr. har nyligen lanserat en ny kollektion av damkläder som syftar till att möta både stil och komfort, vilket har väckt intresse bland modeintresserade i regionen.
['"Heby-textil']
[{'entity': {'"Heby-textil'}, 'description': '"Bolaget skall bedriva tillverkning och försäljning inom textil- och plastbranscherna och idka därmed förenlig verksamhet samt förvärva och förvalta fast och lös egendom."', 'source': 'custom'}]
Heby-textil, som fokuserar på tillverkning och försäljning inom textil- och plastbranscherna, har nyligen ökat sin produktion för att möta den växande efterfrågan på hållbara material.
['"Rukun Invest ']
[{'entity': {'"Rukun Invest '}, 'description': '"Aktiebolagets verksamhet ska vara att driva HVB-hem, hålla föreläsningar om migrations- och integrationsfrågor och göra utvärderingar samt dä

In [26]:
for i in sentences: print(i["sentence"])

CCB Mode eftr. har nyligen lanserat en ny kollektion av damkläder som syftar till att möta både stil och komfort, vilket har väckt intresse bland modeintresserade i regionen.
Heby-textil, som fokuserar på tillverkning och försäljning inom textil- och plastbranscherna, har nyligen ökat sin produktion för att möta den växande efterfrågan på hållbara material.
Rukun Invest har nyligen meddelat att de planerar att utöka sin verksamhet inom HVB-hem samt intensifiera sina föreläsningar om migrations- och integrationsfrågor.
SRBS Group annonserar idag att de kommer att inleda verksamhet inom finansiell rådgivning riktad mot andra företag.
Motor Halland har nyligen utökat sin bildäcksverksamhet för att möta den ökande efterfrågan på däckservice i regionen.
Gärdin & Persson har nyligen utökat sitt sortiment av byggmaterial och verktyg för att möta den ökande efterfrågan inom bygg- och anläggningsbranschen.
Björnö Maskin har under det senaste året ökat sin verksamhet inom uthyrning av jordbruksm

In [27]:
sentences_df = pd.DataFrame(sentences)  
sentences_df.to_csv('./sentences_df.csv', index=False)